In [1]:
import os
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
today = date.today().strftime("%Y%m%d")
import gc

### Preliminar functions and imports

In [2]:
def customer_category_attribution(x):
    if x==1:
        return 'One-timer'
    if x==2:
        return 'Two-timer'
    else:
        return 'Recurring'

In [3]:
def flatten (table):
    if type(table.columns)==pd.MultiIndex:
        columns_to_look = [name_tmp for name_tmp in table.columns]

        columns_df = [ str(t[0])+'_'+str(t[1]) for t in columns_to_look]
        columns_df.insert(0,table.index.name)

        df = pd.DataFrame(columns = columns_df)

        index = 0
        for i in table.index:
            row = [table[r][i] for r in columns_to_look]
            row.insert(0,i)
            df.loc[index] = row
            index = index + 1
        return(df)
    else :
        table = pd.DataFrame(table)
        table.reset_index(level=0, inplace=True)
        return table

In [4]:
df_articles = pd.read_csv('./Vertbaudet/France/src/Original/Articles/Export_FR_Articles_Julie.csv',
                                     usecols = ["DEPARTEMENT_NIV2","CODSAI","REFSTK", "TOP_MN_Max"])

df_codsoc = pd.read_csv('./Vertbaudet/France/src/20210921_CODSOC.csv')

df_client_dept = pd.read_csv('./20210923_Export_tab_FR/20210921_INDIVIDUS.tab', sep = '\t',
                 usecols = ['IDIND', 'CODPOST', "DATPREMCDEWEB"])

clients_mag = pd.read_csv("./Vertbaudet/France/src/20211004_UNIQUE_IDIND_TICKET.csv",sep=",")

df_client = pd.read_csv('./Vertbaudet/France/Output_old/iris_med_rev_clients.csv', usecols = ['IDIND', 'Mediane'])

df_IRIS_by_dep = pd.read_excel('./Vertbaudet/France/INSEE_data/INSEE_IRIS_by_dep.xlsx', usecols = ['CODGEO','MED18'])

C:\Users\pierrick\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
gc.collect()

258

# Import Client base and create fields

In [6]:
gc.collect()
df = pd.DataFrame()
for year in range(2017,2022):
    print(year)
    gc.collect()
    df_tmp = pd.read_csv('./Vertbaudet/France/src/Original/Lignes_commandes/Export_FR_LIGNECOMMANDE_'+str(year)+'.csv', 
                              encoding = 'unicode_escape', 
                              usecols = ['IDIND', 'IDCLI', 'IDFOYER', 'DATCDE', 'DN', 'DBI', 'QTE', 'NUMCDE', 'CODSAI','REFSTK'])
    
    ### df_evt_tclub
    df_evt_tclub = pd.read_csv('C:/Users/pierrick/eleven/Engagements - Vertbaudet/3. Data received/3. Extract/1. France/20211022_TYPEVT_'+str(year)[2:]+'.csv')
    df_tmp = df_tmp.merge(df_evt_tclub, how = 'left', on = ['IDCLI', 'DATCDE', 'REFSTK', 'CODSAI', 'NUMCDE'])
    del df_evt_tclub
    gc.collect()
    
    
    ### Merge with article table to get DEPARTEMENT_NIV2
    df_tmp = df_tmp.merge(df_articles, how = 'left', on = ["CODSAI","REFSTK"])
    
    ### Replace $null$
    df_tmp = df_tmp.replace({'$null$': np.nan})
    ### Clean IDIND index 
    df_tmp.IDIND = df_tmp.IDIND.astype(float)
    df_tmp.IDFOYER = df_tmp.IDFOYER.astype(float)
    
    ### Filter mag clients
    df_tmp = df_tmp[~df_tmp.IDIND.isin(clients_mag.IDIND)]

    ### Filter non-french command with CODSOC
    df_tmp = df_tmp.merge(df_codsoc, how = 'left', on = 'NUMCDE')
    df_tmp = df_tmp[df_tmp.CODSOC==0]

    #### Merge to get Mediane rev
    df_tmp = df_tmp.merge(df_client, how = 'left', on = 'IDIND')

    ### add departments and cohort
    df_tmp = df_tmp.merge(df_client_dept.replace({'$null$': np.nan}), how = 'left', on = 'IDIND')
    df_tmp = df_tmp[df_tmp.DATPREMCDEWEB != '$null$']
    df_tmp['DATCDE'] = pd.to_datetime(df_tmp["DATCDE"])
    df_tmp['DATPREMCDEWEB'] = pd.to_datetime(df_tmp["DATPREMCDEWEB"])
    #df_tmp['DEPARTEMENT'] = df_tmp['CODPOST'].apply(lambda x : str(x)[:2])
    df_tmp['COHORT'] = pd.to_datetime(df_tmp["DATPREMCDEWEB"]).dt.year
    
    ### filter cohort
    df_tmp = df_tmp[(df_tmp['COHORT'] >= 2017) & (df_tmp['COHORT'] < 2021)]
    
    ### filter on DN <= DB
    df_tmp = df_tmp[df_tmp.DN <= df_tmp.DBI]
    df_tmp = df_tmp[df_tmp.DN >= 0]

    df = pd.concat([df,df_tmp[['IDIND', 'DN', 'DBI', 'NUMCDE', 'DATCDE', 'DATPREMCDEWEB', 'DEPARTEMENT_NIV2', 'QTE', 'Mediane', 'COHORT', 'TOP_MN_Max', 'TYPEVT', 'TCLUB']]])
  
    del df_tmp
    gc.collect()

2017


C:\Users\pierrick\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0,2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\pierrick\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


2018


C:\Users\pierrick\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0,2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


2019


C:\Users\pierrick\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0,2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


2020


C:\Users\pierrick\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0,2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


2021


C:\Users\pierrick\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0,2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


- Exclude B2B

In [7]:
# Exclude B2B clients
df_b2b = pd.read_csv('./Vertbaudet/France/src/20211012_TB2B.csv')
df_b2b[['IDIND', 'TB2B']] = df_b2b[['IDIND', 'TB2B']].astype(float)
df = df.merge(df_b2b, how = 'left', on = 'IDIND')
df = df[df['TB2B']==0]
df = df.drop(columns = ['TB2B'])
del df_b2b

- Compte commande

In [8]:
nb_order =  flatten(df.sort_values(by='DATCDE',ascending=True).groupby(['NUMCDE']).agg({'IDIND':'first', 'DATCDE': 'first'}))
nb_order['ORDER_NUMBER'] = nb_order.sort_values(by='DATCDE',ascending=True).groupby(['IDIND']).cumcount()+1
df = pd.merge(df,nb_order[['NUMCDE','ORDER_NUMBER']],on='NUMCDE',how='left')

- Client Category

In [9]:
### Add CLIENT_CATEGORY
nb_commande = flatten(df.groupby(['IDIND']).agg({'NUMCDE':pd.Series.nunique}))
nb_commande.columns = ['IDIND','NB_ORDERS']
df = pd.merge(df,nb_commande,on='IDIND',how='left',suffixes=(False,False))
df['CLIENT_CATEGORY'] = df.NB_ORDERS.apply(lambda x: customer_category_attribution(x))

- New Department

In [10]:
### Create NEW_DEP field

def dif(x,y):
    return len(y.difference(x))

df_list_dept_by_cmd = df[df['ORDER_NUMBER'].isin(range(1,3))].groupby(['IDIND', 'ORDER_NUMBER']).agg({'ORDER_NUMBER': 'first', 'DEPARTEMENT_NIV2': lambda x: list(pd.Series.unique(x))})
df_cmd_1_2 = df_list_dept_by_cmd[df_list_dept_by_cmd['ORDER_NUMBER']==1].merge(df_list_dept_by_cmd[df_list_dept_by_cmd['ORDER_NUMBER']==2], how = 'left', on = 'IDIND').rename(columns = {'DEPARTEMENT_NIV2_x': 'DEPART_1', 'DEPARTEMENT_NIV2_y': 'DEPART_2'})
df_cmd_1_2 = df_cmd_1_2[(df_cmd_1_2.DEPART_1.notna()) & (df_cmd_1_2.DEPART_2.notna())]
df_cmd_1_2['NEW_DEP'] = df_cmd_1_2.apply(lambda x: dif(set(x.DEPART_1), set(x.DEPART_2)), axis = 1)
df = df.merge(df_cmd_1_2['NEW_DEP'], how = 'left', left_on = 'IDIND', right_index = True)

del df_cmd_1_2
del df_list_dept_by_cmd
gc.collect()

0

- Churn Delay

In [12]:
### Add churn_delay field
df_churn = df.sort_values(by = 'DATCDE', ascending = True).groupby('IDIND').agg({'DATCDE': 'last', 'DATPREMCDEWEB': 'last', 'IDIND':'last'})
df_second = df[df['ORDER_NUMBER'] == 2].drop_duplicates('IDIND')[['IDIND', 'DATCDE']].rename(columns = {'DATCDE': 'DATCDE_2'})
df_churn = df_churn.merge(df_second, how = 'left', left_index = True,  right_on = 'IDIND')
df_churn = df_churn.reset_index()
df_churn['CHURN_DELAY'] = (df_churn['DATCDE'] + timedelta(days=365)) - df_churn['DATPREMCDEWEB']
df_churn['CHURN_DELAY_2'] = (df_churn['DATCDE'] + timedelta(days=365)) - df_churn['DATCDE_2']
df_churn = df_churn.drop(columns = 'index')

df = df.merge(df_churn[['IDIND','CHURN_DELAY_2', 'CHURN_DELAY']], how = 'left', left_on = 'IDIND', right_on  = 'IDIND')

del df_churn
del df_second

df['CHURN_DELAY'] = df['CHURN_DELAY'].apply(lambda x : x.days)
df['CHURN_DELAY_2'] = df['CHURN_DELAY_2'].apply(lambda x : x.days)

gc.collect()



0

- Discount category

In [38]:
def categorize_distribution_discount(x):
    if x==0.:
        return('No discount')
    if (x>0) & (x<=0.13):
        return('0-13%')
    if (x>0.13) & (x<=0.16):
        return('13-16%')
    if (x>0.16) & (x<=0.3):
        return('16-30%')
    if (x>0.3) & (x<=0.5):
        return('30-50%')
    if (x>0.5) & (x<=0.7):
        return('50-70%')
    if (x>0.7) & (x<=0.9):
        return('70-90%')
    if x>0.9:
        return('>90%')
    
### Discount by article    
df['DISCOUNT'] = df.DBI - df.DN
df['DISCOUNT_PCT'] = 1 - df.DN/df.DBI
df['DISCOUNT_CAT'] = df["DISCOUNT_PCT"].apply(lambda x: categorize_distribution_discount(x))

gc.collect()

114

In [40]:
### Add DISCOUNT_CMD_CAT
df_discount_cmd = df.groupby('NUMCDE').agg({'DN': sum, 'DBI':sum})
df_discount_cmd['DISCOUNT_CMD_PCT'] = 1 - (df_discount_cmd.DN / df_discount_cmd.DBI)
df_discount_cmd['DISCOUNT_CMD_CAT'] = df_discount_cmd["DISCOUNT_CMD_PCT"].apply(lambda x: categorize_distribution_discount(x))
df = df.merge(df_discount_cmd['DISCOUNT_CMD_CAT'], how = 'left', on = 'NUMCDE')

gc.collect()

23

- Departement commande

In [41]:
### Add DEPARTEMENT_CMD field
cat_order = df.groupby(['NUMCDE', 'DEPARTEMENT_NIV2']).agg({'DN': sum, 'DEPARTEMENT_NIV2': 'first'})
cat_order = cat_order.sort_values(by='DN',ascending=False).groupby('NUMCDE').agg({'DEPARTEMENT_NIV2': 'first'})
cat_order = cat_order.rename(columns = {'DEPARTEMENT_NIV2': 'DEPARTEMENT_CMD'})

df = pd.merge(df,cat_order, how='left', on='NUMCDE', right_index = False,  suffixes=(False,False))

del cat_order
gc.collect()

15

- Recruitment department

In [43]:
df_recruit = df[df.ORDER_NUMBER == 1][['IDIND', 'DEPARTEMENT_CMD']].drop_duplicates('IDIND').rename(columns = {'DEPARTEMENT_CMD': 'DEPARTEMENT_FIRST'})
df = df.merge(df_recruit, how = 'left', on = 'IDIND')

In [44]:
gc.collect()

132

- Commande exclusivement MN

In [63]:
df_mn = df.groupby('NUMCDE')[['NUMCDE', 'TOP_MN_Max']].agg({'TOP_MN_Max': min}).rename(columns = {'TOP_MN_Max': 'CMD_MN'})
df = df.merge(df_mn, how = 'left', on = 'NUMCDE')

- MN recruit

In [159]:
df_mn_recruit = df[df.ORDER_NUMBER == 1][['IDIND', 'CMD_MN']].drop_duplicates('IDIND').rename(columns = {'CMD_MN': 'MN_FIRST'})
df = df.merge(df_mn_recruit, how = 'left', on = 'IDIND')

- LTV

In [77]:
import datetime
from datetime import datetime, time ,date ,timedelta

# Add DAT_12 and DAT_14 column
df['DAT_12'] = df['DATPREMCDEWEB'] + timedelta(days=365)
df['DAT_24'] = df['DATPREMCDEWEB'] + timedelta(days=730)

### Add LTV_12 column
df_ltv_12 = df[df['DATCDE']<=df['DAT_12']][['IDIND', 'DN']]
df_ltv_12 = df_ltv_12.groupby('IDIND').agg({'DN': sum})
df_ltv_12 = df_ltv_12.rename(columns = {'DN': 'LTV_12'})
print(df_ltv_12['LTV_12'].mean())
df = df.merge(df_ltv_12, how = 'left', on = 'IDIND')

del df_ltv_12

### Add LTV_24 column
df_ltv_24 = df[df['DATCDE']<=df['DAT_24']][['IDIND', 'DN']]
df_ltv_24 = df_ltv_24.groupby('IDIND').agg({'DN': sum})
df_ltv_24 = df_ltv_24.rename(columns = {'DN': 'LTV_24'})
print(df_ltv_24['LTV_24'].mean())
df = df.merge(df_ltv_24, how = 'left', on = 'IDIND')

del df_ltv_24

gc.collect()

121.9110836696581
150.33917873961332


64

In [98]:
df['DAT_36'] = df['DATPREMCDEWEB'] + timedelta(days=1095)

### Add LTV_36 column
df_ltv_36 = df[df['DATCDE']<=df['DAT_36']][['IDIND', 'DN']]
df_ltv_36 = df_ltv_36.groupby('IDIND').agg({'DN': sum})
df_ltv_36 = df_ltv_36.rename(columns = {'DN': 'LTV_36'})
print(df_ltv_36['LTV_36'].mean())
df = df.merge(df_ltv_36, how = 'left', on = 'IDIND')

del df_ltv_36

gc.collect()

168.64949830975934


6026

- Enfants

In [119]:
enfants = pd.read_csv(os.path.join("C:/Users/pierrick/eleven/Engagements - Vertbaudet/3. Data received/3. Extract/1. France/20211018_ENFANT.csv"),sep=",")

C:\Users\pierrick\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [90]:
enfants = pd.read_csv(os.path.join("C:/Users/pierrick/eleven/Engagements - Vertbaudet/3. Data received/3. Extract/1. France/20211018_ENFANT.csv"),sep=",")
df_b2b = pd.read_csv('./Vertbaudet/France/src/20211012_TB2B.csv')

enfants["IDIND"] = enfants["IDIND"].astype(float)
enfants = enfants.merge(df_b2b, on="IDIND", how='left')
enfants = enfants[enfants.TB2B == 0.]
enfants['DATNAIS'] = pd.to_datetime(enfants['DATNAIS'])
nb_enfant = flatten(enfants.groupby('IDIND').agg({'DATNAIS': 'count'}))
nb_enfant.columns = ['IDIND', 'NB_ENFANTS']

eldest_child = flatten(enfants.groupby('IDIND').agg({'DATNAIS': 'min'}))
eldest_child.columns = ['IDIND', 'DATNAIS_OLDEST']
enfants = enfants.merge(nb_enfant, on='IDIND', how='left')
enfants = enfants.merge(eldest_child, on='IDIND', how='left')

enfants = enfants[enfants.TINFOREEL == 1.]
enfants = enfants[enfants.NB_ENFANTS == 1]
df = df.merge(enfants[['IDIND', 'NB_ENFANTS', 'DATNAIS_OLDEST']], on='IDIND', how='left')
df['AGE_ENFANT_ACHAT_YEARS'] = np.floor((df.DATCDE - df.DATNAIS_OLDEST).dt.days/365)
df['AGE_ENFANT_ACHAT_MONTHS'] = np.floor((df.DATCDE - df.DATNAIS_OLDEST).dt.days/30.437)

cat_age = df.sort_values(by='DATCDE',ascending=True).drop_duplicates(subset='IDIND', keep='first', inplace=False)[['IDIND','AGE_ENFANT_ACHAT_YEARS', 'AGE_ENFANT_ACHAT_MONTHS']]
cat_age.columns = ['IDIND','AGE_ENFANT_ACHAT_FIRST_YEARS','AGE_ENFANT_ACHAT_FIRST_MONTHS']
df = pd.merge(df,cat_age,on='IDIND',how='left',suffixes=(False,False))

- Acquisition channel

In [97]:
df_code_tracking = pd.read_csv('./Vertbaudet/France/src/20210928_CodeTracking.tab', sep = '\t')

df_input_code_tracking = pd.read_excel('./Vertbaudet/France/src/INPUT_CODES_TRACKING.xlsx', sheet_name = 'CODES_TRACKING')

df_code_tracking = df_code_tracking.merge(df_input_code_tracking, how = 'left', on = 'CODE_TRACKING')

df = df.merge(df_code_tracking[['NUMCDE', 'LEVIER', 'NEW_LEVIER']], how = 'left', on = 'NUMCDE')

In [174]:
df[df.NEW_LEVIER.isin(['COMPARATEURS', 'GOOGLE SHOPPING', 'AFFILIATION'])].drop_duplicates('IDIND').groupby('NEW_LEVIER').agg({'CHURN_DELAY': 'mean', 'LTV_12': 'mean', 'LTV_24': 'mean'}).to_excel('./Vertbaudet/France/Output/'+today+'_churn_ltv_bad_acq_channel.xlsx')

In [176]:
df[df.NEW_LEVIER.isin(['COMPARATEURS', 'GOOGLE SHOPPING', 'AFFILIATION'])].drop_duplicates('IDIND').groupby(['NEW_LEVIER', 'CLIENT_CATEGORY']).agg({'IDIND': pd.Series.nunique}).unstack().to_excel('./Vertbaudet/France/Output/'+today+'_client_cat_bad_acq_channel.xlsx')

- Segmentation fréquence

In [13]:
def segmentation_small_freq(x):
    if x < 7:
        return '< 1 week'
    elif x < 14:
        return '1-2 weeks'
    elif x < 21:
        return '2-3 weeks'
    elif x < 30:
        return '3-4 weeks'
    elif x < 90:
        return '1-3 months'
    elif x < 180:
        return '3-6 months'
    elif x < 270:
        return '6-9 months'
    elif x < 365 :
        return '9-12 months'
    elif x > 365 :
        return '> 12 months'

In [14]:
def segmentation_intraweek(x):
    if x < 1:
        return '< 1 day'
    elif x < 2:
        return '1-2 days'
    elif x < 3:
        return '2-3 days'
    elif x < 4:
        return '3-4 days'
    elif x < 5:
        return '4-5 days'
    elif x < 6:
        return '5-6 days'
    elif x < 7:
        return '6-7 days'
    else:
        return '>= 1 week'

In [16]:
df_first = df[df['ORDER_NUMBER'] == 1].drop_duplicates('IDIND')[['IDIND', 'DATCDE']]

df_second = df[df['ORDER_NUMBER'] == 2].drop_duplicates('IDIND')[['IDIND', 'DATCDE']].rename(columns = {'DATCDE': 'DATCDE_2'})

df_first = df_first.merge(df_second, how = 'left', on = 'IDIND')

df_first['GAP'] = (df_first['DATCDE_2'] - df_first['DATCDE']).dt.days

### Sanity check
print(df_first[df_first.GAP < 0].shape)

#df_first['FREQ_CAT_INTRAWEEK'] = df_first['GAP'].apply(segmentation_intraweek)
df_first['FREQ_CAT_SMALL'] = df_first['GAP'].apply(segmentation_small_freq)

#df = df.merge(df_first[['IDIND', 'FREQ_CAT_INTRAWEEK']], how = 'left', on = 'IDIND')
df = df.merge(df_first[['IDIND', 'FREQ_CAT_SMALL']], how = 'left', on = 'IDIND')



del df_first
del df_second
gc.collect()  

(15, 4)


15

- Add RETOUR field

In [17]:
def map_retour(x):
    if (x == 7) or (x == 15):
        return 1
    else:
        return 0

df['RETOUR'] = df['TYPEVT'].apply(map_retour)

In [18]:
def map_retour_sec(x):
    if x == 7 :
        return 1
    else:
        return 0

df['RETOUR_SEC'] = df['TYPEVT'].apply(map_retour_sec)

# % RETOUR by REORDER DELAY

In [20]:
df_retour_by_freq_cat_small = df[df['DATCDE']==df['DATPREMCDEWEB']].sort_values(by = 'RETOUR', ascending = False).drop_duplicates(['IDIND', 'NUMCDE']).groupby(['FREQ_CAT_SMALL', 'RETOUR']).agg({'IDIND': pd.Series.nunique})

In [30]:
df_retour_by_freq_cat_small.unstack().to_excel('./Vertbaudet/France/Output/'+today+'_retour_freq_cat_small.xlsx')

In [33]:
df[df['DATCDE']==df['DATPREMCDEWEB']].sort_values(by = 'RETOUR_SEC', ascending = False).drop_duplicates(['IDIND', 'NUMCDE']).groupby(['FREQ_CAT_SMALL', 'RETOUR_SEC']).agg({'IDIND': pd.Series.nunique}).unstack().to_excel('./Vertbaudet/France/Output/'+today+'_retour_sec_freq_cat_small.xlsx')

In [42]:
df[df['DATCDE']==df['DATPREMCDEWEB']].sort_values(by = 'RETOUR_SEC', ascending = False).drop_duplicates(['IDIND', 'NUMCDE']).groupby(['FREQ_CAT_SMALL', 'TYPEVT']).agg({'IDIND': pd.Series.nunique}).unstack().to_excel('./Vertbaudet/France/Output/'+today+'_typevt_freq_cat_small.xlsx')

In [51]:
df[df['DATCDE']==df['DATPREMCDEWEB']].sort_values(by = 'RETOUR_SEC', ascending = False).drop_duplicates(['IDIND', 'NUMCDE']).groupby(['FREQ_CAT_INTRAWEEK', 'RETOUR_SEC']).agg({'IDIND': pd.Series.nunique}).unstack().to_excel('./Vertbaudet/France/Output/'+today+'_retour_sec_freq_cat_intraweek.xlsx')

# % TCLUB

In [22]:
df['TCLUB'] = df['TCLUB'].astype(bool)

In [35]:
df.drop_duplicates(['IDIND', 'NUMCDE']).groupby('TCLUB').agg({'NUMCDE': pd.Series.nunique}).to_excel('./Vertbaudet/France/Output/'+today+'_tclub.xlsx')

# Two-timers / Freq cat

In [38]:
df[(df['CLIENT_CATEGORY'] == 'Two-timer') & (df['DATCDE']==df['DATPREMCDEWEB'])].groupby(['FREQ_CAT_SMALL', 'RETOUR_SEC']).agg({'IDIND': pd.Series.nunique}).unstack().to_excel('./Vertbaudet/France/Output/'+today+'two_timer_retour_freq_cat_small.xlsx')

In [39]:
df.groupby(['RETOUR_SEC', 'FREQ_CAT_SMALL']).agg({'IDIND': pd.Series.nunique}).unstack().to_excel('./Vertbaudet/France/Output/'+today+'freq_cat_small_by_retour.xlsx')

In [52]:
df.groupby(['FREQ_CAT_INTRAWEEK', 'CLIENT_CATEGORY']).agg({'IDIND': pd.Series.nunique}).unstack().to_excel('./Vertbaudet/France/Output/'+today+'cat_cli_by_freq_cat_intraweek.xlsx')

### Client ever been club

In [74]:
df_club = df.sort_values(by = 'TCLUB', ascending = False).groupby('IDIND').agg({'TCLUB': 'first'}).rename(columns = {'TCLUB': 'CLUB_EVER'})
df = df.merge(df_club, how = 'left', on = 'IDIND')

In [77]:
df_club_by_freq_cat = df.groupby(['FREQ_CAT_SMALL', 'CLUB_EVER']).agg({'IDIND': pd.Series.nunique})

In [78]:
df_club_by_client_cat = df.groupby(['CLIENT_CATEGORY', 'CLUB_EVER']).agg({'IDIND': pd.Series.nunique})

In [81]:
df_club_by_freq_cat.unstack().to_excel('./Vertbaudet/France/Output/'+today+'club_by_freq_cat_small.xlsx')

In [82]:
df_club_by_client_cat.unstack().to_excel('./Vertbaudet/France/Output/'+today+'club_by_client_cat.xlsx')

### By freq cat and cat client, % of TCLUB command

In [23]:
df_tclub_cde_by_feq_cat_small = df.drop_duplicates(subset = ['IDIND', 'NUMCDE']).groupby(['FREQ_CAT_SMALL', 'TCLUB']).agg({'NUMCDE': pd.Series.nunique})

In [25]:
df_tclub_cde_by_feq_cat_small.unstack().to_excel('./Vertbaudet/France/Output/'+today+'_df_tclub_cde_by_feq_cat_small.xlsx')

In [34]:
df_tclub_cde_by_client_cat = df.drop_duplicates(subset = ['IDIND', 'NUMCDE']).groupby(['CLIENT_CATEGORY', 'TCLUB']).agg({'NUMCDE': pd.Series.nunique})

In [35]:
df_tclub_cde_by_client_cat.unstack().to_excel('./Vertbaudet/France/Output/'+today+'_df_tclub_cde_by_cat_client.xlsx')